In [ ]:
from datetime import datetime
import locale
import os

from dotenv import load_dotenv
import pandas as pd
import sqlalchemy as sa

load_dotenv()

locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")
locale.setlocale(locale.LC_MONETARY, "pt_BR.UTF-8")

pd.set_option("display.float_format", lambda x: locale.currency(val=x, symbol=False, grouping=True))

engine: sa.Engine = sa.engine.create_engine(os.getenv("URL_MYSQL"))

In [ ]:
releases: list[dict[str: str]] = [
    # {"nm_curso": "..."},
]

rows_inserted: int = pd.DataFrame(releases).to_sql(name="lançamentos", con=engine, if_exists="append", index=False)
print(f"Foram {rows_inserted} registros inseridos com sucesso.")

In [ ]:
pd.read_sql(sql=sa.text("SELECT id_lançamento AS Código, lançamento AS Lançamento FROM lançamento"), con=engine)

In [ ]:
paychecks: list[dict[str: int | bool | float]] = [
    # {"id_lançamento": 36, "período": 202407, "acerto": True, "valor": -84.31},
    # {"id_lançamento": 40, "período": 202407, "acerto": True, "valor": -22.4},
    # {"id_lançamento": 1, "período": 202407, "acerto": False, "valor": 4523.22},
    # {"id_lançamento": 2, "período": 202407, "acerto": False, "valor": 1258.46},
    # {"id_lançamento": 26, "período": 202407, "acerto": False, "valor": 2649.73},
    # {"id_lançamento": 38, "período": 202407, "acerto": False, "valor": -321.23},
    # {"id_lançamento": 42, "período": 202407, "acerto": False, "valor": -590.19},
    # {"id_lançamento": 43, "período": 202407, "acerto": False, "valor": -168.62},
    # {"id_lançamento": 48, "período": 202407, "acerto": False, "valor": -463.71},
    # {"id_lançamento": 50, "período": 202407, "acerto": False, "valor": -908.86},
    # {"id_lançamento": 53, "período": 202407, "acerto": False, "valor": -859.75},
]

row_paychecks: int = pd.DataFrame(paychecks).to_sql(name="espelho", con=engine, if_exists="append", index=False)
print(f"Foram {row_paychecks} registros inseridos com sucesso.")

In [ ]:
period: int = datetime.now().year * 100 + datetime.now().month

stmt: str = f"""
    SELECT
        l.lançamento,
        e.período,
        IF(e.acerto, 'acerto', 'mês') AS espelho,
        e.valor
    FROM
        espelho e
        INNER JOIN lançamento l ON e.id_lançamento = l.id_lançamento
    WHERE
        e.período = {period}
    ORDER BY
        e.acerto DESC
"""

df_mes: pd.DataFrame = pd.read_sql(sql=sa.text(stmt), con=engine)
df_mes["período"] = pd.to_datetime(df_mes["período"], format="%Y%m").dt.strftime("%B de %Y")
df_mes

In [ ]:
ano: int = datetime.now().year

stmt: str = f"""
    SELECT
        l.lançamento,
        e.período,
        IF(e.acerto = 1, 'acerto', 'mês') AS acerto,
        e.valor
    FROM
        espelho e
        INNER JOIN lançamento l ON e.id_lançamento = l.id_lançamento
    WHERE
        e.período LIKE '{ano}%'
"""

df_ano: pd.DataFrame = pd.read_sql(sql=sa.text(stmt), con=engine)
df_ano = df_ano.pivot(values=["valor"], index=["lançamento", "acerto"], columns=["período"])
df_ano.columns = df_ano.columns.droplevel(level=0)
df_ano.reset_index(inplace=True)
df_ano.fillna(value=0, inplace=True)
df_ano.sort_values(["acerto"], inplace=True, ignore_index=True)
df_ano.columns.rename("", inplace=True)
df_ano["média"] = df_ano.mean(axis=1, numeric_only=True)
df_ano["total"] = df_ano[df_ano.columns[:-1]].sum(axis=1, numeric_only=True)
df_ano.loc["sumário"] = df_ano.sum(numeric_only=True)
df_ano.fillna(value="", inplace=True)
df_ano.iloc[-1, 0] = "Sumário"
df_ano.set_index(["lançamento"], inplace=True)
df_ano.rename(columns={
    int(f"{ano}01"): "jan", int(f"{ano}02"): "fev", int(f"{ano}03"): "mar", int(f"{ano}04"): "abr",
    int(f"{ano}05"): "mai", int(f"{ano}06"): "jun", int(f"{ano}07"): "jul", int(f"{ano}08"): "ago",
    int(f"{ano}09"): "set", int(f"{ano}10"): "out", int(f"{ano}11"): "nov", int(f"{ano}12"): "dez",
}, inplace=True)
df_ano

In [ ]:
stmt: str = """
    SELECT
        SUBSTR(período, 1, 4) AS ano,
        CONCAT('mês ', SUBSTR(período, 5)) AS mes,
        SUM(valor) AS valor
    FROM
        espelho
    GROUP BY
        ano,
        mes
"""

df_anuais: pd.DataFrame = pd.read_sql(sql=stmt, con=engine)
df_anuais = df_anuais.pivot(columns=["mes"], index=["ano"], values=["valor"])
df_anuais.columns = df_anuais.columns.droplevel(level=0)
df_anuais.reset_index(inplace=True)
df_anuais["ano"] = df_anuais["ano"].astype(int)
df_anuais.set_index(["ano"], inplace=True)
df_anuais.fillna(0, inplace=True)
df_anuais["média"] = df_anuais.mean(axis=1)
df_anuais["total"] = df_anuais[df_anuais.columns[:-1]].sum(axis=1)
df_anuais.columns = ["jan", "fev", "mar", "abr", "mai", "jun", "jul", "ago", "set", "out", "nov", "dez", "média", "total"]
df_anuais

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ano: int = datetime.now().year

plt.figure(figsize=(16, 6))
plt.style.use("ggplot")

ax: plt.Axes = sns.barplot(data=df_anuais.loc[[ano], df_anuais.columns[:-2]])
ax.set_title(f"Espelho {ano}", loc="center", fontweight="bold", fontsize=12)
ax.set(xlabel="", ylabel="", yticks=[])

for mes in range(12):
    ax.bar_label(ax.containers[mes], fmt=lambda i: locale.currency(val=i, symbol=False, grouping=True), fontsize=10)

plt.show()